In [2]:
from scrape_courses import get_courses_from_dgscene, get_readable_course_name
import json

ModuleNotFoundError: No module named 'logger'

In [1]:
# TODO
# generate list of all courses
# write to json
courses = get_courses_from_dgscene()
course_names = {
    course: get_readable_course_name(course) for course in courses
}
with open('data/course_names.json', 'w') as f:
    json.dump(course_names, f, indent=4)

NameError: name 'get_courses_from_dgscene' is not defined

In [9]:
print(get_readable_course_name('Hillcrest_Country_Club_Golf_Course_with_27_Hole_Te'))

SyntaxError: cannot use absolute path on element (<string>)